In [ ]:
import hopsworks
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from hsml.model_schema import ModelSchema
from hsml.schema import Schema
import joblib

In [ ]:
project = hopsworks.login()

In [ ]:
fs = project.get_feature_store()
heart_fg = fs.get_feature_group(name="heart", version=1)

# Get Data from Feature Store


In [ ]:
heart_fg = fs.get_feature_group(name="heart", version=1)

In [ ]:
query = heart_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="heart",
                                  version=1,
                                  description="Read from Heart dataset",
                                  labels=["heartdisease"],
                                  query=query)

# Train Test Split and Scale

In [ ]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

In [ ]:
y_test = y_test.to_numpy()[:, 0]
y_train = y_train.to_numpy()[:, 0]

In [ ]:
# TODO : REMOVE CELL

heart_df = pd.read_csv("./dataset/heart_res.csv")

from sklearn.model_selection import train_test_split

target = heart_df['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(heart_df.drop('HeartDisease', axis=1), target, test_size=0.2, random_state=42)

In [ ]:
heart_df = pd.read_csv("./dataset/heart_res.csv")

from sklearn.model_selection import train_test_split

target = heart_df['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(heart_df.drop('HeartDisease', axis=1), target, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

In [ ]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'iris_model' directory will be saved to the model registry. Create the dir, first.
model_dir="heart_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model, model_dir + "/heart_model.pkl")

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
heart_model = mr.python.create_model(
    name="heart_model", 
    # model_schema=model_schema,
    description="heart Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
heart_model.save(model_dir)